1. My gut reaction was theat introspection would not be very helpful for thinking about AI, but as I consulted my limited knowledge bank of AI (a bit of introspection), I realize it is used a lot for machine learning. If you can inform an AI what it's goals are and teach it how to adjust it's approach to meet it's goal, and teach it how to reflect on it's actions (introspection), then it can improve itself. In machine learning, I know they let they AI preform linear regression to try and minimize the margin of error for a given problem.

It certainly cannot be the only thing that informs our development of AI, but when it comes to teaching the AI to improve itself, it's a no brainer.

In [ ]:
from tools.aima.search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule
from random import randint
import copy

class State():
    def __init__(self, numCities, map):
        self.map = map
        self.path =[]
        # Initialize a full state solution
        for i in range( numCities ):
            self.path.append(i)

    def swap(self, action):
        temp = self.path[action[0]]
        self.path[action[0]] = self.path[action[1]]
        self.path[action[1]] = temp

    def calculateDistance(self):
        distance = 0
        for i in range( 1, len( self.path ) ):
            distance += self.map[ self.path[i] ][ self.path[i - 1] ]
        return distance


class TSP(Problem):
    """
    State: x value for the abs function variant f(x)
    Move: a new x value delta steps from the current x (in both directions)
    """

    def __init__(self, numCities):
        self.initial = State( numCities, self.createMap( numCities ) )

    def actions(self, state):
        possibleActions = []
        # Find ten possible swaps
        for i in range(10):
            rand1 = randint( 0, len( state.path ) - 1 )
            rand2 = randint( 0, len( state.path ) - 1 )
            if rand1 != rand2:
                possibleActions.append( [ rand1, rand2 ])
        return possibleActions

    def result(self, state, action):
        newState = copy.deepcopy( state )
        newState.swap( action )
        return newState

    def value(self, state):
        return -1 * state.calculateDistance()

    def createMap(self, numCities):

        map = []
        # Initialize numCites x numCities array
        for i in range(numCities):
            map.append([None] * numCities)
        # Fill in half of the matrix while mirror across the middle axis
        for x in range(numCities):
            for y in range(numCities):
                if x >= y:
                    continue
                if x < y:
                    dist = randint(0, 100)
                    map[x][y] = dist
                    map[y][x] = dist
        # Print the map for fact checking
        for y in range(numCities):
            for x in range(numCities):
                print(map[x][y], end="\t\t")
            print()

        return map


if __name__ == '__main__':

    p = TSP(10)
    # Solve the problem using hill-climbing.
    hill_solution = hill_climbing(p)
    print('Hill-climbing solution       Path: ' + str(hill_solution.path)
          + '\tvalue: ' + str(p.value(hill_solution))
          )

    # Solve the problem using simulated annealing.
    annealing_solution = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )

    print('Simulated annealing solution Path: ' + str( annealing_solution.path )
          + '\tvalue: ' + str(p.value(annealing_solution))
          )

My state is a complete state solution with storing the path and the adjacency matrix for the graph. Each action represents swapping two nodes such that a new path is created. It is limited to 10 random swaps to avoid the search space from becoming too large.

Simmulated annealing almost always outpreforms a raw hill climbing approach. This is because blindly going forward often is good for the next node visit, but that node is a long way away from other nodes.

In [ ]:
from csp import  parse_neighbors, CSP, min_conflicts


def printResult( res ):
    for state in res:
        print( state + ": " + res[state] )


def schedule_constraint(varA, domA, varB, domB, recurse=0):
    if varA == varB and domA == domB:
        return False

    # Parse strings into arrays
    # [0] : Prof, [1] : Time, [2] : Room
    domAList = domA.split(',')
    domBList = domB.split(',')

    # Professor cannot teach two classes at the same time
    if domAList[0] == domBList[0] and domAList[1] == domBList[1]:
        return False

    # A room cannot be double occupied
    if domAList[2] == domBList[2] and domAList[1] == domBList[1]:
        return False

    # Specific constraints
    if varA == 'cs108' and domAList[0] != 'DS':
        return False

    if varA == 'cs112' and domAList[0] != 'JA':
        return False

    if recurse == 0:
        return schedule_constraint(varB, domB, varA, domA, 1)

    return True


profs = 'KVL, JA, DS, HP,'.split()
times = '800MWF, 900MWF, 1030MWF, 1130MWF, 830TT, 1030TT,'.split()
rooms = 'NH253 SB382'.split()
domainsList = []
# Create every permutation of professor, time, and room
for p in profs:
    for t in times:
        for r in rooms:
            key = p + t + r
            domainsList.append( key )

variables = ['cs108', 'cs112', 'cs104', 'cs212', 'cs214', 'cs262', 'cs232']
domains = {}
# Assign each class to all the domain permutations
for var in variables:
    domains[var] = domainsList
neighbors = parse_neighbors("""cs108: DS,800MWF,NH253""", variables)
for type in [domainsList, variables]:
    for A in type:
        for B in type:
            if A != B:
                if B not in neighbors[A]:
                    neighbors[A].append(B)
                if A not in neighbors[B]:
                    neighbors[B].append(A)

problem = CSP(variables, domains, neighbors, schedule_constraint)

result = min_conflicts(problem, max_steps=1000)

print("Min conflicts:")
printResult( result)